In [15]:
# Imports
import gymnasium as gym
import matplotlib.pyplot as plt
import torch
from torch import nn
import random
from typing import Literal, Any
import numpy as np


In [2]:
# torch settings
device: Literal['cuda'] = "cuda"
torch.set_grad_enabled(False)
torch.set_default_tensor_type("torch.cuda.FloatTensor")


In [3]:
# LilGuy def
class LilGuy(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.fc: nn.Sequential = nn.Sequential(
            nn.Linear(4, 16, device=device),
            nn.ReLU(),
            nn.Linear(16, 2, device=device),
            nn.Softmax(dim=0)
        )
        self.reward: float = 0.

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        return self.fc(inputs)

    def run(self, env: gym.Env) -> float:
        self.reward = 0
        obs = env.reset()[0]
        action: int
        term: bool = False
        trunc: bool = False
        total_reward: float = 0
        curr_reward: float
        while not (term or trunc):
            action = int(torch.argmax(self(torch.tensor(obs))))
            obs, curr_reward, term, trunc = env.step(action)[:-1]
            total_reward += curr_reward
        return total_reward

    def run_avg(self, env: gym.Env, runs: int = 3) -> float:
        rewards: list[float] = []
        for _ in range(runs):
            rewards.append(self.run(env))
        self.reward = sum(rewards)/len(rewards)
        return self.reward

    def __gt__(self, other) -> bool: return self.reward > other.reward
    def __ge__(self, other) -> bool: return self.reward >= other.reward
    def __lt__(self, other) -> bool: return self.reward < other.reward
    def __le__(self, other) -> bool: return self.reward <= other.reward


In [20]:
def init_weights(m: nn.Module) -> None:
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        m.bias.data.fill_(0)


def init_agent() -> LilGuy:
    g: LilGuy = LilGuy()
    g.apply(init_weights)
    return g


def init_agents(n: int) -> list[LilGuy]:
    return [init_agent() for i in range(n)]


In [5]:
def generate_generation(gen: list[LilGuy], num_parents: int, mutation_power: float, env: gym.Env) -> list[LilGuy]:
    parents: list[LilGuy] = sorted(gen, reverse=True)[:num_parents]
    for i in parents[:5]:
        i.run_avg(env)
    parents = sorted(parents, reverse=True)
    new_gen: list[LilGuy] = [LilGuy() for _ in gen]
    new_gen[0] = parents[0]
    for i in new_gen[1:]:
        curr_parent: LilGuy = random.choice(parents)
        new_params: torch.Tensor = nn.utils.parameters_to_vector(
            curr_parent.parameters())
        new_params += mutation_power * torch.rand_like(new_params)  # MUTATION
        nn.utils.vector_to_parameters(new_params, i.parameters())
    return new_gen


In [11]:
# main
POP_SIZE: int = 50
GENS: int = 10
PARENT_SIZE: int = 5
MUTATION_POWER: float = .02
agents: list[LilGuy] = init_agents(POP_SIZE)
env: gym.Env = gym.make("CartPole-v1")
for i in range(GENS):
    for i in agents:
        i.run_avg(env)
    print([i.reward for i in sorted(agents, reverse=True)])
    agents = generate_generation(agents, PARENT_SIZE, MUTATION_POWER, env)


[367.3333333333333, 181.66666666666666, 160.66666666666666, 158.0, 124.33333333333333, 118.33333333333333, 104.0, 100.33333333333333, 79.0, 73.0, 43.666666666666664, 42.333333333333336, 32.333333333333336, 20.666666666666668, 17.666666666666668, 16.0, 14.0, 11.333333333333334, 11.333333333333334, 11.0, 10.333333333333334, 10.0, 10.0, 10.0, 9.666666666666666, 9.666666666666666, 9.666666666666666, 9.666666666666666, 9.666666666666666, 9.666666666666666, 9.666666666666666, 9.666666666666666, 9.666666666666666, 9.666666666666666, 9.666666666666666, 9.333333333333334, 9.333333333333334, 9.333333333333334, 9.333333333333334, 9.333333333333334, 9.333333333333334, 9.333333333333334, 9.0, 9.0, 9.0, 9.0, 9.0, 8.666666666666666, 8.666666666666666, 8.666666666666666]
[500.0, 500.0, 443.0, 431.3333333333333, 384.0, 357.0, 349.6666666666667, 348.0, 328.6666666666667, 286.0, 256.3333333333333, 253.33333333333334, 240.0, 233.0, 212.0, 206.33333333333334, 203.66666666666666, 202.0, 154.0, 136.333333333

In [13]:
a: LilGuy = max(agents)
print([i for i in a.parameters()])
env = gym.make("CartPole-v1", render_mode="human")
for _ in range(5): print(a.run(env))
env.close()

[Parameter containing:
tensor([[ 0.2542,  0.0881, -0.1894,  0.1535],
        [-0.1038,  0.3650,  0.6026,  0.1776],
        [ 0.3981, -0.1214,  0.2787,  0.1811],
        [ 0.4900, -0.7035,  0.3484, -0.1392],
        [-0.0247,  0.4488,  0.0942,  0.6481],
        [-0.2666,  0.0989,  0.5573,  0.7555],
        [ 0.3700, -0.3621, -0.3394, -0.2255],
        [-0.0636,  0.6265, -0.4378, -0.3594],
        [-0.2433, -0.1408,  0.1613, -0.3019],
        [ 0.0852,  0.3980,  0.3645,  0.0589],
        [ 0.0314,  0.0982, -0.0733, -0.2856],
        [ 0.2855,  0.1043,  0.4048, -0.1492],
        [ 0.0755,  0.0974,  0.0741,  0.1471],
        [-0.0418,  0.1594, -0.2832,  0.2303],
        [ 0.4863,  0.1013,  0.3151,  0.0596],
        [-0.2360, -0.1307, -0.4785,  0.1807]], requires_grad=True), Parameter containing:
tensor([0.0757, 0.0774, 0.0506, 0.0749, 0.0699, 0.0837, 0.0734, 0.0882, 0.1148,
        0.0651, 0.0581, 0.1404, 0.0848, 0.0832, 0.1131, 0.0895],
       requires_grad=True), Parameter containing:
te